In [2]:
import sklearn.datasets
import pandas as pd
import numpy as np

import sklearn.model_selection
from sklearn.metrics import r2_score, accuracy_score
#from imodels import FIGSRegressor, FIGSClassifier
#from imodels.importance import RandomForestPlusRegressor

import openml
from ucimlrepo import fetch_ucirepo 

from figs_d import *
from fourierDistill import *

ModuleNotFoundError: No module named 'l0learn'

In [ ]:
def traverse_paths(node):
    if node is None:
        return []

    paths = []
    vals = []

    def dfs(current, path, from_left=False):
        if current.left is None and current.right is None and not from_left:
            paths.append(path.copy())
            vals.append(current.value.item())
            return 
        
        if current.left:
            dfs(current.left, [], from_left=True)

        if current.right:
            path.append(current)
            dfs(current.right, path)

    dfs(node, [])

    return list(zip(paths, vals))

In [ ]:
miami_housing = openml.datasets.get_dataset(43093)
X, y, _, _ = miami_housing.get_data(target=miami_housing.default_target_attribute, dataset_format="dataframe")

In [ ]:
figs = FIGSRegressor(max_rules = 20, max_trees = 10)
figs.fit(X, y, feature_names = list(X.columns))

In [ ]:
figs_rules = [traverse_paths(t) for t in figs.trees_]

In [ ]:
figs_rules = [
    x
    for xs in figs_rules
    for x in xs
]

In [ ]:
figs_rules